In [42]:
import pandas as pd
import numpy as np

# Import Dataset

Import the preprocessed data set

In [43]:
df_boston = pd.read_csv('./data/prepared/rides_data_prepared.csv', index_col=0, dtype={'start_station_id': np.int64, 'end_station_id': 'string', 'end_station_name': 'string', 'start_station_name': 'string', 'bike_id': np.int64, 'user_type': 'string'})
df_boston['start_time'] = pd.to_datetime(df_boston['start_time'], format='%Y-%m-%d %X')
 
df_boston.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,trip_length,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,date_time,max_temp,min_temp,precip
0,2015-01-01 00:21:44,2015-01-01 00:30:47,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,277,Subscriber,543.0,42.387995,-71.119084,42.373379,-71.111075,2015-01-01 00:00:00,NaN,NaN,NaN
1,2015-01-01 00:27:03,2015-01-01 00:34:21,80,95,MIT Stata Center at Vassar St / Main St,Cambridge St - at Columbia St / Webster Ave,648,Subscriber,438.0,42.361962,-71.092053,42.372969,-71.094445,2015-01-01 00:00:00,NaN,NaN,NaN
2,2015-01-01 00:31:31,2015-01-01 00:35:46,91,68,One Kendall Square at Hampshire St / Portland St,Central Square at Mass Ave / Essex St,555,Subscriber,255.0,42.366277,-71.091690,42.365070,-71.103100,2015-01-01 01:00:00,NaN,NaN,NaN
3,2015-01-01 00:53:46,2015-01-01 01:00:58,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,1307,Subscriber,432.0,42.387995,-71.119084,42.373379,-71.111075,2015-01-01 01:00:00,NaN,NaN,NaN
4,2015-01-01 01:07:06,2015-01-01 01:19:21,105,88,Lower Cambridgeport at Magazine St/Riverside Rd,Inman Square at Vellucci Plaza / Hampshire St,177,Customer,735.0,42.356954,-71.113687,42.374035,-71.101427,2015-01-01 01:00:00,NaN,NaN,NaN


# Demand Calculation

The demand on an hourly basis is defined by the number of rentals started within the respective hour, e.g a rental with start time 10.30 and end time 11.30 counts as a demand of the hour 10


In [44]:
df_demand = df_boston.resample(rule='H', on='start_time').size().reset_index(name='demand')
df_demand.columns = ['date_time', 'demand']
df_demand.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  8760 non-null   datetime64[ns]
 1   demand     8760 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 137.0 KB


In [45]:
df_weather = pd.read_csv('./data/weather_hourly_boston.csv')
df_weather['date_time'] = pd.to_datetime(df_weather['date_time'], format='%Y-%m-%d %X')
df_demand = df_demand.merge(df_weather, how = 'left', on = 'date_time')
df_demand.head(10)

,date_time,demand,max_temp,min_temp,precip
0,2015-01-01 00:00:00,4,NaN,NaN,NaN
1,2015-01-01 01:00:00,3,NaN,NaN,NaN
2,2015-01-01 02:00:00,1,NaN,NaN,NaN
3,2015-01-01 03:00:00,0,NaN,NaN,NaN
4,2015-01-01 04:00:00,1,NaN,NaN,NaN
5,2015-01-01 05:00:00,0,NaN,NaN,NaN
6,2015-01-01 06:00:00,0,-5.6,-5.6,0.0
7,2015-01-01 07:00:00,0,-5.6,-5.6,0.0
8,2015-01-01 08:00:00,4,-5.6,-5.6,0.0
9,2015-01-01 09:00:00,2,-4.4,-4.4,0.0


# Feature Engineering

## Previous demand as input

In [50]:
df_demand['demand_h-1'] = df_demand.demand.shift(periods=11)
df_demand['demand_h-2'] = df_demand.demand.shift(periods=2)
df_demand['demand_h-24'] = df_demand.demand.shift(periods=24)
df_demand.head(50)

,date_time,demand,max_temp,min_temp,precip,demand_h-1,demand_h-2,demand_h-24
0,2015-01-01 00:00:00,4,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 01:00:00,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01 02:00:00,1,NaN,NaN,NaN,NaN,4.0,NaN
3,2015-01-01 03:00:00,0,NaN,NaN,NaN,NaN,3.0,NaN
4,2015-01-01 04:00:00,1,NaN,NaN,NaN,NaN,1.0,NaN
5,2015-01-01 05:00:00,0,NaN,NaN,NaN,NaN,0.0,NaN
6,2015-01-01 06:00:00,0,-5.6,-5.6,0.0,NaN,1.0,NaN
7,2015-01-01 07:00:00,0,-5.6,-5.6,0.0,NaN,0.0,NaN
8,2015-01-01 08:00:00,4,-5.6,-5.6,0.0,NaN,0.0,NaN
9,2015-01-01 09:00:00,2,-4.4,-4.4,0.0,NaN,0.0,NaN


## Construct weather features
### Rain feature

In [52]:
df_demand['rain'] = df_demand.apply((lambda row: 0 if row.precip == 0 else 1), axis=1)
df_demand.head()

,date_time,demand,max_temp,min_temp,precip,demand_h-1,demand_h-2,demand_h-24,rain
0,2015-01-01 00:00:00,4,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2015-01-01 01:00:00,3,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2015-01-01 02:00:00,1,NaN,NaN,NaN,NaN,4.0,NaN,1
3,2015-01-01 03:00:00,0,NaN,NaN,NaN,NaN,3.0,NaN,1
4,2015-01-01 04:00:00,1,NaN,NaN,NaN,NaN,1.0,NaN,1
